HDP-base spell-sound analyzer
developed by Kow Kuroda (kow.kuroda@gmail.com)

History
2024/08/19 Cython を使った cy_gen_ngrams を利用可能にした
2024/09/02 n-gram 生成の generic function を導入し，コードを簡略化

In [50]:
#!pip install -U pyLDAvis
#!pip install -U pandas

In [51]:
## imports
import os, sys
import pprint as pp
import unicodedata

In [52]:
## 一つ上の階層のファイルを見るように設定
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

In [53]:
## Cython module の生成 (必要に応じて)
#!python clean setup.py build_ext --inplace

In [54]:
## Cython を使うかどうか
use_Cython = True
if use_Cython:
    %load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [55]:
## term settings
term_classes        = [ 'spell', 'sound' ]
term_class          = term_classes[0]
ngram_is_inclusive  = True
ngram_inclusiveness = 2
## doc settings
max_doc_size        = 11
min_doc_size        =  3
print(f"max_doc_size: {max_doc_size}")
print(f"min_doc_size: {min_doc_size}")
### boundary handling
add_boundary        = False
boundary_mark       = "#"
## term setting
seg_joint           = ""
gap_mark            = "…"
term_is_skippy      = True
n_for_ngram         = 4
max_gap_ratio       = 0.8
max_gap_size        = round(max_doc_size * max_gap_ratio)
print(f"term_class: {term_class}")
print(f"term_is_skippy: {term_is_skippy}")
print(f"max_gap_size: {max_gap_size}")
print(f"n_for_ngram: {n_for_ngram}")
### accent handling
suppress_accents    = True
accent_marks        = [ "ˈ", "ˌ" ] 
if term_class == 'sound':
    if suppress_accents:
        accent_status = "-unaccented"
    else:
        accent_stratus = "-accented"
else:
    accent_status = ""
print(f"accent_status: {accent_status}")
## define term_type
if term_class == 'spell':
    if term_is_skippy:
        term_type = f"sp_skippy{n_for_ngram}gram"
    else:
        term_type = f"sp_{n_for_ngram}gram"
else:
    if term_is_skippy:
        term_type = f"sn_skippy{n_for_ngram}gram"
    else:
        term_type = f"sn_{n_for_ngram}gram"
## check
print(f"term_type: {term_type}")

max_doc_size: 11
min_doc_size: 3
term_class: spell
term_is_skippy: True
max_gap_size: 9
n_for_ngram: 4
accent_status: 
term_type: sp_skippy4gram


In [56]:
## target language
## a key must be part of a file name 
target_lang_dict = {    'en_US' : 'English (US)',
                        'en_UK' : 'English (UK)',
                        'en_N_only' : 'English noun (WN)',
                        'en_V_only' : 'English verb (WN)',
                        'en_A_only' : 'English adj (WN)',
                        'en_R_only' : 'English adv (WN)',
                        'ar'    : 'Arabic',
                        'de'    : 'German',
                        'de_N_only' : 'German Nouns',
                        'de_non_N_only' : 'German Non-nouns',
                        'eo'    : 'Esperanto',
                        'es_ES' : 'Spanish (Spain)',
                        'es_MX' : 'Spanish (Mexico)',
                        'fi'    : 'Finnish',
                        'fr_FR' : 'French (France)',
                        'fr_QC' : 'French (Quebec)',
                        'is'    : 'Icelandic',
                        'ir'    : 'Irish',
                        'nl'    : 'Dutch',
                        'ro'    : 'Romanian',
                        'sw'    : 'Swahili' }
## proper language selection
target_lang_keys = [    'en_US', # 0
                        'en_UK', # 1
                        'en_N_only', # 2
                        'en_V_only', # 3
                        'en_A_only', # 4
                        'en_R_only', # 5
                        'ar', # 6
                        'de', # 7
                        'de_N_only', # 8
                        'de_non_N_only', # 9
                        'eo', 'es_ES', 'es_MX',
                        'fi', 'fr_FR', 'fr_QC',
                        'is', 'nl', 'ro', 'sw',
                        'ir' # This lacks sound
                    ]
## check
target_lang_key  = target_lang_keys[4]
print(f"target_lang_key: {target_lang_key}")
print(f"target lang: {target_lang_dict[target_lang_key]} [{target_lang_key}]")
## target_attr [effective only for Irish]
target_class = ""
#target_class = None # This causes an unrediable error
if target_lang_key == "ir":
    target_classes = [ 'adjectives', 'nouns', 'verbs' ]
    target_class = f"-{target_classes[3]}"
print(f"target_class: {target_class}")

target_lang_key: en_A_only
target lang: English adj (WN) [en_A_only]
target_class: 


In [57]:
## LDA/HDP
apply_term_filtering = True
## The following parameters need to be relatively large to prevent "Row sum not equal 1" error
term_minfreq       = 2
## The following value is crucial to prevent "Row sum not equal 1" error
min_bot_size       = min_doc_size # 3
abuse_threshold    = 0.05 # larger value selects shorter units, smaller value selects longer units
# number of terms listed for a given topic
n_docs_to_show     = 15
n_terms_to_show    = 20
n_terms_to_save    = 120

In [58]:
## sampling
source_sampling          = True
source_sampling_rate     = 0.5
source_sampling_max_size = 5000
second_sampling          = False
second_sampling_rate     = 0.7

In [59]:
## set target files
import glob
data_dir1     = "data/open-dict-ipa/data1"
data_dir2     = "data/open-dict-ipa/data1a"
data_dir3     = "data/wn3"
data_dir4     = "data/irish"
target_files  = glob.glob(f"{data_dir1}/*")
target_files2 = glob.glob(f"{data_dir2}/*")
target_files.extend(target_files2)
target_files3 = glob.glob(f"{data_dir3}/*")
target_files.extend(target_files3)
target_files4 = glob.glob(f"{data_dir4}/*")
target_files.extend(target_files4)
#
target_files = sorted([ file for file in target_files if ".csv" in file ])
pp.pprint(target_files)

['data/irish/word-irish-adjectives-spell.csv',
 'data/irish/word-irish-noun-phrases-spell.csv',
 'data/irish/word-irish-nouns-spell.csv',
 'data/irish/word-irish-possessives-spell.csv',
 'data/irish/word-irish-prepositions-spell.csv',
 'data/irish/word-irish-verbs-spell.csv',
 'data/open-dict-ipa/data1/ar.csv.gz',
 'data/open-dict-ipa/data1/de.csv.gz',
 'data/open-dict-ipa/data1/en_UK.csv.gz',
 'data/open-dict-ipa/data1/en_US.csv.gz',
 'data/open-dict-ipa/data1/eo.csv.gz',
 'data/open-dict-ipa/data1/es_ES.csv.gz',
 'data/open-dict-ipa/data1/es_MX.csv.gz',
 'data/open-dict-ipa/data1/fa.csv.gz',
 'data/open-dict-ipa/data1/fi.csv.gz',
 'data/open-dict-ipa/data1/fr_FR.csv.gz',
 'data/open-dict-ipa/data1/fr_QC.csv.gz',
 'data/open-dict-ipa/data1/is.csv.gz',
 'data/open-dict-ipa/data1/ja.csv.gz',
 'data/open-dict-ipa/data1/jam.csv.gz',
 'data/open-dict-ipa/data1/ma.csv.gz',
 'data/open-dict-ipa/data1/nb.csv.gz',
 'data/open-dict-ipa/data1/nl.csv.gz',
 'data/open-dict-ipa/data1/or.csv.gz',
 '

In [60]:
## get source data from files
import pandas as pd
import gzip
#target_language_key = "en_US" # can be changed to get other languages
#if target_class != "" or target_class is not None:
if target_class != "":
    target_file = [ f for f in target_files if target_lang_key in f and target_class in f ][0]
else:
    target_file = [ f for f in target_files if target_lang_key in f ][0]
print(f"processing: {target_file}")
##
if target_lang_key == "ir":
    col_names = ['spell', 'POS']
else:
    col_names = ['spell', 'sound']
#
if target_file.endswith(".gz"):
    with gzip.open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
else:
    with open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
## normalize characters
raw_df['spell'] = raw_df['spell'].apply(lambda x: unicodedata.normalize('NFC', str(x)))
## modify sound
try:
    sounds = raw_df['sound'].apply(lambda x: x.strip('/') )
    sounds = [ x.split("/,")[0] for x in sounds ] # picks up only the first of multiple entries
    raw_df['sound'] = sounds
except (AttributeError, KeyError):
    pass
#
raw_df.sample(10)

processing: data/wn3/en_A_only.csv


,spell,sound
416,askance,əˈskæns
3684,obligated,ˈɑbɫəˌɡeɪtɪd
4942,stiff,ˈstɪf
336,aphrodisiac,ˌæfɹoʊˈdiziæk
2595,hot,ˈhɑt
5526,unlawful,ənˈɫɔfəɫ
2740,inadvisable,ˌɪnædˈvaɪzəbəɫ
3896,peachy,ˈpitʃi
5884,wrongful,ˈɹɔŋfəɫ
5587,unrealistic,ənˌɹiˈɫɪstɪk


In [61]:
## source sampling
len(raw_df)
if source_sampling:
	print(f"source sampling applied")
	if len(raw_df) >= source_sampling_max_size:
		raw_df = raw_df.sample(source_sampling_max_size)
	else:
		raw_df = raw_df.sample(round(len(raw_df) * source_sampling_rate))
## remove accent marking
if suppress_accents:
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: "".join([ y for y in list(x) if y not in accent_marks ]))
	except KeyError:
		pass
## add boudary marks
if add_boundary:
	raw_df['spell'] = raw_df['spell'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	except KeyError:
		pass
#
print(raw_df)

source sampling applied
                spell            sound
1078       coincident      koʊɪnsədənt
5846         windward          wɪndwɝd
2386         gorgeous          ɡɔɹdʒəs
3953  pharmacological  fɑɹməkəɫɑdʒɪkəɫ
1325         covalent        koʊvɑɫənt
...               ...              ...
5164      theological      θiəɫɑdʒɪkəɫ
497         available        əveɪɫəbəɫ
3581            nervy             nɝvi
330         antiviral       æntivaɪɹəɫ
4276   quintessential     kwɪntɪsɛnʃəɫ

[5000 rows x 2 columns]


In [62]:
## generate 1-grams for spell and sound
## spell
raw_df['sp_1gram'] = raw_df['spell'].apply(lambda x: list(str(x)))
# add column of size
raw_df['sp_size'] = raw_df['sp_1gram'].apply(lambda x: len(x))
# add column of count of '-' inside
raw_df['hyphen'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("-"))
# add column of count of '.' inside
raw_df['period'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("."))
## sound
# takes the first entry, removes '/' around
try:
    raw_df['sn_1gram'] = raw_df['sound'].apply(lambda x: list(x) )
except (TypeError, KeyError):
    pass
# add column of size
try:
    raw_df['sn_size'] = raw_df['sn_1gram'].apply(lambda x: len(x))
except KeyError:
    pass
## check
raw_df

,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
1078,coincident,koʊɪnsədənt,"[c, o, i, n, c, i, d, e, n, t]",10,0,0,"[k, o, ʊ, ɪ, n, s, ə, d, ə, n, t]",11
5846,windward,wɪndwɝd,"[w, i, n, d, w, a, r, d]",8,0,0,"[w, ɪ, n, d, w, ɝ, d]",7
2386,gorgeous,ɡɔɹdʒəs,"[g, o, r, g, e, o, u, s]",8,0,0,"[ɡ, ɔ, ɹ, d, ʒ, ə, s]",7
3953,pharmacological,fɑɹməkəɫɑdʒɪkəɫ,"[p, h, a, r, m, a, c, o, l, o, g, i, c, a, l]",15,0,0,"[f, ɑ, ɹ, m, ə, k, ə, ɫ, ɑ, d, ʒ, ɪ, k, ə, ɫ]",15
1325,covalent,koʊvɑɫənt,"[c, o, v, a, l, e, n, t]",8,0,0,"[k, o, ʊ, v, ɑ, ɫ, ə, n, t]",9
...,...,...,...,...,...,...,...,...
5164,theological,θiəɫɑdʒɪkəɫ,"[t, h, e, o, l, o, g, i, c, a, l]",11,0,0,"[θ, i, ə, ɫ, ɑ, d, ʒ, ɪ, k, ə, ɫ]",11
497,available,əveɪɫəbəɫ,"[a, v, a, i, l, a, b, l, e]",9,0,0,"[ə, v, e, ɪ, ɫ, ə, b, ə, ɫ]",9
3581,nervy,nɝvi,"[n, e, r, v, y]",5,0,0,"[n, ɝ, v, i]",4
330,antiviral,æntivaɪɹəɫ,"[a, n, t, i, v, i, r, a, l]",9,0,0,"[æ, n, t, i, v, a, ɪ, ɹ, ə, ɫ]",10


In [63]:
## filtering raw_data by size
print(f"term_type: {term_type}")
if "sp_" in term_type:
    df_filtered = raw_df[ (raw_df['sp_size'] <= max_doc_size) & (raw_df['sp_size'] >= min_doc_size) & (raw_df['hyphen'] == 0) & (raw_df['period'] == 0) ]
else:
    df_filtered = raw_df[ (raw_df['sn_size'] <= max_doc_size) & (raw_df['sn_size'] >= min_doc_size) ]
#
df_filtered

term_type: sp_skippy4gram


,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
1078,coincident,koʊɪnsədənt,"[c, o, i, n, c, i, d, e, n, t]",10,0,0,"[k, o, ʊ, ɪ, n, s, ə, d, ə, n, t]",11
5846,windward,wɪndwɝd,"[w, i, n, d, w, a, r, d]",8,0,0,"[w, ɪ, n, d, w, ɝ, d]",7
2386,gorgeous,ɡɔɹdʒəs,"[g, o, r, g, e, o, u, s]",8,0,0,"[ɡ, ɔ, ɹ, d, ʒ, ə, s]",7
1325,covalent,koʊvɑɫənt,"[c, o, v, a, l, e, n, t]",8,0,0,"[k, o, ʊ, v, ɑ, ɫ, ə, n, t]",9
2973,invalid,ɪnvəɫəd,"[i, n, v, a, l, i, d]",7,0,0,"[ɪ, n, v, ə, ɫ, ə, d]",7
...,...,...,...,...,...,...,...,...
2930,interesting,ɪntɝəstɪŋ,"[i, n, t, e, r, e, s, t, i, n, g]",11,0,0,"[ɪ, n, t, ɝ, ə, s, t, ɪ, ŋ]",9
5164,theological,θiəɫɑdʒɪkəɫ,"[t, h, e, o, l, o, g, i, c, a, l]",11,0,0,"[θ, i, ə, ɫ, ɑ, d, ʒ, ɪ, k, ə, ɫ]",11
497,available,əveɪɫəbəɫ,"[a, v, a, i, l, a, b, l, e]",9,0,0,"[ə, v, e, ɪ, ɫ, ə, b, ə, ɫ]",9
3581,nervy,nɝvi,"[n, e, r, v, y]",5,0,0,"[n, ɝ, v, i]",4


In [64]:
## define df after second sampling if any
len(df_filtered)
if second_sampling:
    df = df_filtered.sample(round(len(df_filtered) * second_sampling_rate))
else:
    df = df_filtered
len(df)

4453

In [65]:
## generic function for n-gram generation
def add_ngram_to_df(dfx, n_for_ngram: int, var_prefix: str = "", ngram_is_skippy: bool = False, ngram_is_inclusive: bool = ngram_is_inclusive, ngram_inclusiveness: int = ngram_inclusiveness, seg_joint: str = "", use_Cython: bool = False, check: bool = False):
    """
    generic function for adding n-gram column to df with a specified n for ngram
    """
    inclusion_size = (n_for_ngram - ngram_inclusiveness)
    print(f"inclusion_size: {inclusion_size}")
    assert inclusion_size >= 0
    source_var = f"{var_prefix}1gram"
    print(f"source_var: {source_var}")
    unigrams = df[source_var]
    if use_Cython:
        import cy_gen_ngrams
        if ngram_is_skippy:
            ngrams = [ [seg_joint.join(x) for x in cy_gen_ngrams.cy_gen_skippy_ngrams(x, n = n_for_ngram, check = False)] for x in unigrams ]
        else:
            ngrams = [ [seg_joint.join(x) for x in cy_gen_ngrams.cy_gen_ngrams(x, n = n_for_ngram, check = False)] for x in unigrams ]
    else:
        import gen_ngrams
        if ngram_is_skippy:
            ngrams = [ gen_ngrams.gen_skippy_ngrams(x, n = n_for_ngram, sep = "", check = False) for x in unigrams ]
        else:
            ngrams = [ gen_ngrams.gen_ngrams(x, n = n_for_ngram, sep = "", check = False) for x in unigrams ]
    ## 包括的 2gramの生成
    if ngram_is_inclusive:
        assert (n_for_ngram - 1) > 0
        if ngram_is_skippy and n_for_ngram > 2:
            supplement_var = f"{var_prefix}skippy{n_for_ngram - 1}gram"
        else:
            supplement_var = f"{var_prefix}{n_for_ngram - 1}gram"
        print(f"supplement_var: {supplement_var}")
        for i, g in enumerate(ngrams):
            included = [ x for x in list(dfx[supplement_var])[i] if len(x) >= inclusion_size ]
            if len(included) > 0:
                g.extend(included)
    ## 変数の追加
    if ngram_is_skippy:
        added_var = f"{var_prefix}skippy{n_for_ngram}gram"
    else:
        added_var = f"{var_prefix}{n_for_ngram}gram"
    print(f"added_var: {added_var}")
    dfx[added_var] = ngrams
    ## check result
    print(dfx[added_var])

# Spell

In [66]:
## spell 2grams
add_ngram_to_df(df, n_for_ngram = 2, var_prefix = "sp_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 0
source_var: sp_1gram
supplement_var: sp_1gram
added_var: sp_2gram
1078    [co, oi, in, nc, ci, id, de, en, nt, c, o, i, ...
5846    [wi, in, nd, dw, wa, ar, rd, w, i, n, d, w, a,...
2386    [go, or, rg, ge, eo, ou, us, g, o, r, g, e, o,...
1325    [co, ov, va, al, le, en, nt, c, o, v, a, l, e,...
2973        [in, nv, va, al, li, id, i, n, v, a, l, i, d]
                              ...                        
2930    [in, nt, te, er, re, es, st, ti, in, ng, i, n,...
5164    [th, he, eo, ol, lo, og, gi, ic, ca, al, t, h,...
497     [av, va, ai, il, la, ab, bl, le, a, v, a, i, l...
3581                      [ne, er, rv, vy, n, e, r, v, y]
330     [an, nt, ti, iv, vi, ir, ra, al, a, n, t, i, v...
Name: sp_2gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [67]:
## spell 3grams
add_ngram_to_df(df, n_for_ngram = 3, var_prefix = "sp_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 1
source_var: sp_1gram
supplement_var: sp_2gram
added_var: sp_3gram
1078    [coi, oin, inc, nci, cid, ide, den, ent, co, o...
5846    [win, ind, ndw, dwa, war, ard, wi, in, nd, dw,...
2386    [gor, org, rge, geo, eou, ous, go, or, rg, ge,...
1325    [cov, ova, val, ale, len, ent, co, ov, va, al,...
2973    [inv, nva, val, ali, lid, in, nv, va, al, li, ...
                              ...                        
2930    [int, nte, ter, ere, res, est, sti, tin, ing, ...
5164    [the, heo, eol, olo, log, ogi, gic, ica, cal, ...
497     [ava, vai, ail, ila, lab, abl, ble, av, va, ai...
3581       [ner, erv, rvy, ne, er, rv, vy, n, e, r, v, y]
330     [ant, nti, tiv, ivi, vir, ira, ral, an, nt, ti...
Name: sp_3gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [68]:
## spell 4grams
add_ngram_to_df(df, n_for_ngram = 4, var_prefix = "sp_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 2
source_var: sp_1gram
supplement_var: sp_3gram
added_var: sp_4gram
1078    [coin, oinc, inci, ncid, cide, iden, dent, coi...
5846    [wind, indw, ndwa, dwar, ward, win, ind, ndw, ...
2386    [gorg, orge, rgeo, geou, eous, gor, org, rge, ...
1325    [cova, oval, vale, alen, lent, cov, ova, val, ...
2973    [inva, nval, vali, alid, inv, nva, val, ali, l...
                              ...                        
2930    [inte, nter, tere, eres, rest, esti, stin, tin...
5164    [theo, heol, eolo, olog, logi, ogic, gica, ica...
497     [avai, vail, aila, ilab, labl, able, ava, vai,...
3581          [nerv, ervy, ner, erv, rvy, ne, er, rv, vy]
330     [anti, ntiv, tivi, ivir, vira, iral, ant, nti,...
Name: sp_4gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [69]:
## spell 5grams
add_ngram_to_df(df, n_for_ngram = 5, var_prefix = "sp_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 3
source_var: sp_1gram
supplement_var: sp_4gram
added_var: sp_5gram
1078    [coinc, oinci, incid, ncide, ciden, ident, coi...
5846    [windw, indwa, ndwar, dward, wind, indw, ndwa,...
2386    [gorge, orgeo, rgeou, geous, gorg, orge, rgeo,...
1325    [coval, ovale, valen, alent, cova, oval, vale,...
2973    [inval, nvali, valid, inva, nval, vali, alid, ...
                              ...                        
2930    [inter, ntere, teres, erest, resti, estin, sti...
5164    [theol, heolo, eolog, ologi, logic, ogica, gic...
497     [avail, vaila, ailab, ilabl, lable, avai, vail...
3581                   [nervy, nerv, ervy, ner, erv, rvy]
330     [antiv, ntivi, tivir, ivira, viral, anti, ntiv...
Name: sp_5gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [70]:
## spell skippy 2grams
add_ngram_to_df(df, n_for_ngram = 2, var_prefix = "sp_", ngram_is_skippy = True, ngram_is_inclusive = ngram_is_inclusive, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 0
source_var: sp_1gram
supplement_var: sp_1gram
added_var: sp_skippy2gram
1078    [co, c…i, c…n, c…c, c…i, c…d, c…e, c…n, c…t, o...
5846    [wi, w…n, w…d, w…w, w…a, w…r, w…d, in, i…d, i…...
2386    [go, g…r, g…g, g…e, g…o, g…u, g…s, or, o…g, o…...
1325    [co, c…v, c…a, c…l, c…e, c…n, c…t, ov, o…a, o…...
2973    [in, i…v, i…a, i…l, i…i, i…d, nv, n…a, n…l, n…...
                              ...                        
2930    [in, i…t, i…e, i…r, i…e, i…s, i…t, i…i, i…n, i...
5164    [th, t…e, t…o, t…l, t…o, t…g, t…i, t…c, t…a, t...
497     [av, a…a, a…i, a…l, a…a, a…b, a…l, a…e, va, v…...
3581    [ne, n…r, n…v, n…y, er, e…v, e…y, rv, r…y, vy,...
330     [an, a…t, a…i, a…v, a…i, a…r, a…a, a…l, nt, n…...
Name: sp_skippy2gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [71]:
## spell skippy 3grams
add_ngram_to_df(df, n_for_ngram = 3, var_prefix = "sp_", ngram_is_skippy = True, ngram_is_inclusive = ngram_is_inclusive, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 1
source_var: sp_1gram
supplement_var: sp_skippy2gram
added_var: sp_skippy3gram
1078    [coi, co…n, co…c, co…i, co…d, co…e, co…n, co…t...
5846    [win, wi…d, wi…w, wi…a, wi…r, wi…d, w…nd, w…n…...
2386    [gor, go…g, go…e, go…o, go…u, go…s, g…rg, g…r…...
1325    [cov, co…a, co…l, co…e, co…n, co…t, c…va, c…v…...
2973    [inv, in…a, in…l, in…i, in…d, i…va, i…v…l, i…v...
                              ...                        
2930    [int, in…e, in…r, in…e, in…s, in…t, in…i, in…n...
5164    [the, th…o, th…l, th…o, th…g, th…i, th…c, th…a...
497     [ava, av…i, av…l, av…a, av…b, av…l, av…e, a…ai...
3581    [ner, ne…v, ne…y, n…rv, n…r…y, n…vy, erv, er…y...
330     [ant, an…i, an…v, an…i, an…r, an…a, an…l, a…ti...
Name: sp_skippy3gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [72]:
## spell skippy 4grams
add_ngram_to_df(df, n_for_ngram = 4, var_prefix = "sp_", ngram_is_skippy = True, ngram_is_inclusive = ngram_is_inclusive, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 2
source_var: sp_1gram
supplement_var: sp_skippy3gram
added_var: sp_skippy4gram
1078    [coin, coi…c, coi…i, coi…d, coi…e, coi…n, coi…...
5846    [wind, win…w, win…a, win…r, win…d, wi…dw, wi…d...
2386    [gorg, gor…e, gor…o, gor…u, gor…s, go…ge, go…g...
1325    [cova, cov…l, cov…e, cov…n, cov…t, co…al, co…a...
2973    [inva, inv…l, inv…i, inv…d, in…al, in…a…i, in…...
                              ...                        
2930    [inte, int…r, int…e, int…s, int…t, int…i, int…...
5164    [theo, the…l, the…o, the…g, the…i, the…c, the…...
497     [avai, ava…l, ava…a, ava…b, ava…l, ava…e, av…i...
3581    [nerv, ner…y, ne…vy, n…rvy, ervy, ner, ne…v, n...
330     [anti, ant…v, ant…i, ant…r, ant…a, ant…l, an…i...
Name: sp_skippy4gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [73]:
## spell skippy 5grams
add_ngram_to_df(df, n_for_ngram = 5, var_prefix = "sp_", ngram_is_skippy = True, ngram_is_inclusive = ngram_is_inclusive, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 3
source_var: sp_1gram
supplement_var: sp_skippy4gram
added_var: sp_skippy5gram
1078    [coinc, coin…i, coin…d, coin…e, coin…n, coin…t...
5846    [windw, wind…a, wind…r, wind…d, win…wa, win…w…...
2386    [gorge, gorg…o, gorg…u, gorg…s, gor…eo, gor…e…...
1325    [coval, cova…e, cova…n, cova…t, cov…le, cov…l…...
2973    [inval, inva…i, inva…d, inv…li, inv…l…d, inv…i...
                              ...                        
2930    [inter, inte…e, inte…s, inte…t, inte…i, inte…n...
5164    [theol, theo…o, theo…g, theo…i, theo…c, theo…a...
497     [avail, avai…a, avai…b, avai…l, avai…e, ava…la...
3581    [nervy, nerv, ner…y, ne…vy, n…rvy, ervy, ner, ...
330     [antiv, anti…i, anti…r, anti…a, anti…l, ant…vi...
Name: sp_skippy5gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [74]:
## check result
target_vars = [ 'spell' ]
target_vars.extend([ f"sp_{i}gram" for i in range(1,6) ])
target_vars.extend([ f"sp_skippy{i}gram" for i in range(2,6) ])
df[target_vars]

,spell,sp_1gram,sp_2gram,sp_3gram,sp_4gram,sp_5gram,sp_skippy2gram,sp_skippy3gram,sp_skippy4gram,sp_skippy5gram
1078,coincident,"[c, o, i, n, c, i, d, e, n, t]","[co, oi, in, nc, ci, id, de, en, nt, c, o, i, ...","[coi, oin, inc, nci, cid, ide, den, ent, co, o...","[coin, oinc, inci, ncid, cide, iden, dent, coi...","[coinc, oinci, incid, ncide, ciden, ident, coi...","[co, c…i, c…n, c…c, c…i, c…d, c…e, c…n, c…t, o...","[coi, co…n, co…c, co…i, co…d, co…e, co…n, co…t...","[coin, coi…c, coi…i, coi…d, coi…e, coi…n, coi…...","[coinc, coin…i, coin…d, coin…e, coin…n, coin…t..."
5846,windward,"[w, i, n, d, w, a, r, d]","[wi, in, nd, dw, wa, ar, rd, w, i, n, d, w, a,...","[win, ind, ndw, dwa, war, ard, wi, in, nd, dw,...","[wind, indw, ndwa, dwar, ward, win, ind, ndw, ...","[windw, indwa, ndwar, dward, wind, indw, ndwa,...","[wi, w…n, w…d, w…w, w…a, w…r, w…d, in, i…d, i…...","[win, wi…d, wi…w, wi…a, wi…r, wi…d, w…nd, w…n…...","[wind, win…w, win…a, win…r, win…d, wi…dw, wi…d...","[windw, wind…a, wind…r, wind…d, win…wa, win…w…..."
2386,gorgeous,"[g, o, r, g, e, o, u, s]","[go, or, rg, ge, eo, ou, us, g, o, r, g, e, o,...","[gor, org, rge, geo, eou, ous, go, or, rg, ge,...","[gorg, orge, rgeo, geou, eous, gor, org, rge, ...","[gorge, orgeo, rgeou, geous, gorg, orge, rgeo,...","[go, g…r, g…g, g…e, g…o, g…u, g…s, or, o…g, o…...","[gor, go…g, go…e, go…o, go…u, go…s, g…rg, g…r…...","[gorg, gor…e, gor…o, gor…u, gor…s, go…ge, go…g...","[gorge, gorg…o, gorg…u, gorg…s, gor…eo, gor…e…..."
1325,covalent,"[c, o, v, a, l, e, n, t]","[co, ov, va, al, le, en, nt, c, o, v, a, l, e,...","[cov, ova, val, ale, len, ent, co, ov, va, al,...","[cova, oval, vale, alen, lent, cov, ova, val, ...","[coval, ovale, valen, alent, cova, oval, vale,...","[co, c…v, c…a, c…l, c…e, c…n, c…t, ov, o…a, o…...","[cov, co…a, co…l, co…e, co…n, co…t, c…va, c…v…...","[cova, cov…l, cov…e, cov…n, cov…t, co…al, co…a...","[coval, cova…e, cova…n, cova…t, cov…le, cov…l…..."
2973,invalid,"[i, n, v, a, l, i, d]","[in, nv, va, al, li, id, i, n, v, a, l, i, d]","[inv, nva, val, ali, lid, in, nv, va, al, li, ...","[inva, nval, vali, alid, inv, nva, val, ali, l...","[inval, nvali, valid, inva, nval, vali, alid, ...","[in, i…v, i…a, i…l, i…i, i…d, nv, n…a, n…l, n…...","[inv, in…a, in…l, in…i, in…d, i…va, i…v…l, i…v...","[inva, inv…l, inv…i, inv…d, in…al, in…a…i, in…...","[inval, inva…i, inva…d, inv…li, inv…l…d, inv…i..."
...,...,...,...,...,...,...,...,...,...,...
2930,interesting,"[i, n, t, e, r, e, s, t, i, n, g]","[in, nt, te, er, re, es, st, ti, in, ng, i, n,...","[int, nte, ter, ere, res, est, sti, tin, ing, ...","[inte, nter, tere, eres, rest, esti, stin, tin...","[inter, ntere, teres, erest, resti, estin, sti...","[in, i…t, i…e, i…r, i…e, i…s, i…t, i…i, i…n, i...","[int, in…e, in…r, in…e, in…s, in…t, in…i, in…n...","[inte, int…r, int…e, int…s, int…t, int…i, int…...","[inter, inte…e, inte…s, inte…t, inte…i, inte…n..."
5164,theological,"[t, h, e, o, l, o, g, i, c, a, l]","[th, he, eo, ol, lo, og, gi, ic, ca, al, t, h,...","[the, heo, eol, olo, log, ogi, gic, ica, cal, ...","[theo, heol, eolo, olog, logi, ogic, gica, ica...","[theol, heolo, eolog, ologi, logic, ogica, gic...","[th, t…e, t…o, t…l, t…o, t…g, t…i, t…c, t…a, t...","[the, th…o, th…l, th…o, th…g, th…i, th…c, th…a...","[theo, the…l, the…o, the…g, the…i, the…c, the…...","[theol, theo…o, theo…g, theo…i, theo…c, theo…a..."
497,available,"[a, v, a, i, l, a, b, l, e]","[av, va, ai, il, la, ab, bl, le, a, v, a, i, l...","[ava, vai, ail, ila, lab, abl, ble, av, va, ai...","[avai, vail, aila, ilab, labl, able, ava, vai,...","[avail, vaila, ailab, ilabl, lable, avai, vail...","[av, a…a, a…i, a…l, a…a, a…b, a…l, a…e, va, v…...","[ava, av…i, av…l, av…a, av…b, av…l, av…e, a…ai...","[avai, ava…l, ava…a, ava…b, ava…l, ava…e, av…i...","[avail, avai…a, avai…b, avai…l, avai…e, ava…la..."
3581,nervy,"[n, e, r, v, y]","[ne, er, rv, vy, n, e, r, v, y]","[ner, erv, rvy, ne, er, rv, vy, n, e, r, v, y]","[nerv, ervy, ner, erv, rvy, ne, er, rv, vy]","[n

# Sound

In [75]:
## sound 2grams
add_ngram_to_df(df, n_for_ngram = 2, var_prefix = "sn_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 0
source_var: sn_1gram
supplement_var: sn_1gram
added_var: sn_2gram
1078    [ko, oʊ, ʊɪ, ɪn, ns, sə, əd, də, ən, nt, k, o,...
5846        [wɪ, ɪn, nd, dw, wɝ, ɝd, w, ɪ, n, d, w, ɝ, d]
2386        [ɡɔ, ɔɹ, ɹd, dʒ, ʒə, əs, ɡ, ɔ, ɹ, d, ʒ, ə, s]
1325    [ko, oʊ, ʊv, vɑ, ɑɫ, ɫə, ən, nt, k, o, ʊ, v, ɑ...
2973        [ɪn, nv, və, əɫ, ɫə, əd, ɪ, n, v, ə, ɫ, ə, d]
                              ...                        
2930    [ɪn, nt, tɝ, ɝə, əs, st, tɪ, ɪŋ, ɪ, n, t, ɝ, ə...
5164    [θi, iə, əɫ, ɫɑ, ɑd, dʒ, ʒɪ, ɪk, kə, əɫ, θ, i,...
497     [əv, ve, eɪ, ɪɫ, ɫə, əb, bə, əɫ, ə, v, e, ɪ, ɫ...
3581                             [nɝ, ɝv, vi, n, ɝ, v, i]
330     [æn, nt, ti, iv, va, aɪ, ɪɹ, ɹə, əɫ, æ, n, t, ...
Name: sn_2gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [76]:
## sound 3grams
add_ngram_to_df(df, n_for_ngram = 3, var_prefix = "sn_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 1
source_var: sn_1gram
supplement_var: sn_2gram
added_var: sn_3gram
1078    [koʊ, oʊɪ, ʊɪn, ɪns, nsə, səd, ədə, dən, ənt, ...
5846    [wɪn, ɪnd, ndw, dwɝ, wɝd, wɪ, ɪn, nd, dw, wɝ, ...
2386    [ɡɔɹ, ɔɹd, ɹdʒ, dʒə, ʒəs, ɡɔ, ɔɹ, ɹd, dʒ, ʒə, ...
1325    [koʊ, oʊv, ʊvɑ, vɑɫ, ɑɫə, ɫən, ənt, ko, oʊ, ʊv...
2973    [ɪnv, nvə, vəɫ, əɫə, ɫəd, ɪn, nv, və, əɫ, ɫə, ...
                              ...                        
2930    [ɪnt, ntɝ, tɝə, ɝəs, əst, stɪ, tɪŋ, ɪn, nt, tɝ...
5164    [θiə, iəɫ, əɫɑ, ɫɑd, ɑdʒ, dʒɪ, ʒɪk, ɪkə, kəɫ, ...
497     [əve, veɪ, eɪɫ, ɪɫə, ɫəb, əbə, bəɫ, əv, ve, eɪ...
3581                   [nɝv, ɝvi, nɝ, ɝv, vi, n, ɝ, v, i]
330     [ænt, nti, tiv, iva, vaɪ, aɪɹ, ɪɹə, ɹəɫ, æn, n...
Name: sn_3gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [77]:
## sound 4grams
add_ngram_to_df(df, n_for_ngram = 4, var_prefix = "sn_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 2
source_var: sn_1gram
supplement_var: sn_3gram
added_var: sn_4gram
1078    [koʊɪ, oʊɪn, ʊɪns, ɪnsə, nsəd, sədə, ədən, dən...
5846    [wɪnd, ɪndw, ndwɝ, dwɝd, wɪn, ɪnd, ndw, dwɝ, w...
2386    [ɡɔɹd, ɔɹdʒ, ɹdʒə, dʒəs, ɡɔɹ, ɔɹd, ɹdʒ, dʒə, ʒ...
1325    [koʊv, oʊvɑ, ʊvɑɫ, vɑɫə, ɑɫən, ɫənt, koʊ, oʊv,...
2973    [ɪnvə, nvəɫ, vəɫə, əɫəd, ɪnv, nvə, vəɫ, əɫə, ɫ...
                              ...                        
2930    [ɪntɝ, ntɝə, tɝəs, ɝəst, əstɪ, stɪŋ, ɪnt, ntɝ,...
5164    [θiəɫ, iəɫɑ, əɫɑd, ɫɑdʒ, ɑdʒɪ, dʒɪk, ʒɪkə, ɪkə...
497     [əveɪ, veɪɫ, eɪɫə, ɪɫəb, ɫəbə, əbəɫ, əve, veɪ,...
3581                         [nɝvi, nɝv, ɝvi, nɝ, ɝv, vi]
330     [ænti, ntiv, tiva, ivaɪ, vaɪɹ, aɪɹə, ɪɹəɫ, ænt...
Name: sn_4gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [78]:
## sound 5grams
add_ngram_to_df(df, n_for_ngram = 4, var_prefix = "sn_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 2
source_var: sn_1gram
supplement_var: sn_3gram
added_var: sn_4gram
1078    [koʊɪ, oʊɪn, ʊɪns, ɪnsə, nsəd, sədə, ədən, dən...
5846    [wɪnd, ɪndw, ndwɝ, dwɝd, wɪn, ɪnd, ndw, dwɝ, w...
2386    [ɡɔɹd, ɔɹdʒ, ɹdʒə, dʒəs, ɡɔɹ, ɔɹd, ɹdʒ, dʒə, ʒ...
1325    [koʊv, oʊvɑ, ʊvɑɫ, vɑɫə, ɑɫən, ɫənt, koʊ, oʊv,...
2973    [ɪnvə, nvəɫ, vəɫə, əɫəd, ɪnv, nvə, vəɫ, əɫə, ɫ...
                              ...                        
2930    [ɪntɝ, ntɝə, tɝəs, ɝəst, əstɪ, stɪŋ, ɪnt, ntɝ,...
5164    [θiəɫ, iəɫɑ, əɫɑd, ɫɑdʒ, ɑdʒɪ, dʒɪk, ʒɪkə, ɪkə...
497     [əveɪ, veɪɫ, eɪɫə, ɪɫəb, ɫəbə, əbəɫ, əve, veɪ,...
3581                         [nɝvi, nɝv, ɝvi, nɝ, ɝv, vi]
330     [ænti, ntiv, tiva, ivaɪ, vaɪɹ, aɪɹə, ɪɹəɫ, ænt...
Name: sn_4gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [79]:
## sound skippy 2grams
add_ngram_to_df(df, n_for_ngram = 2, var_prefix = "sn_", ngram_is_skippy = True, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 0
source_var: sn_1gram
supplement_var: sn_1gram
added_var: sn_skippy2gram
1078    [ko, k…ʊ, k…ɪ, k…n, k…s, k…ə, k…d, k…ə, k…n, k...
5846    [wɪ, w…n, w…d, w…w, w…ɝ, w…d, ɪn, ɪ…d, ɪ…w, ɪ…...
2386    [ɡɔ, ɡ…ɹ, ɡ…d, ɡ…ʒ, ɡ…ə, ɡ…s, ɔɹ, ɔ…d, ɔ…ʒ, ɔ…...
1325    [ko, k…ʊ, k…v, k…ɑ, k…ɫ, k…ə, k…n, k…t, oʊ, o…...
2973    [ɪn, ɪ…v, ɪ…ə, ɪ…ɫ, ɪ…ə, ɪ…d, nv, n…ə, n…ɫ, n…...
                              ...                        
2930    [ɪn, ɪ…t, ɪ…ɝ, ɪ…ə, ɪ…s, ɪ…t, ɪ…ɪ, ɪ…ŋ, nt, n…...
5164    [θi, θ…ə, θ…ɫ, θ…ɑ, θ…d, θ…ʒ, θ…ɪ, θ…k, θ…ə, θ...
497     [əv, ə…e, ə…ɪ, ə…ɫ, ə…ə, ə…b, ə…ə, ə…ɫ, ve, v…...
3581              [nɝ, n…v, n…i, ɝv, ɝ…i, vi, n, ɝ, v, i]
330     [æn, æ…t, æ…i, æ…v, æ…a, æ…ɪ, æ…ɹ, æ…ə, æ…ɫ, n...
Name: sn_skippy2gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [80]:
## sound skippy 3grams
add_ngram_to_df(df, n_for_ngram = 3, var_prefix = "sn_", ngram_is_skippy = True, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 1
source_var: sn_1gram
supplement_var: sn_skippy2gram
added_var: sn_skippy3gram
1078    [koʊ, ko…ɪ, ko…n, ko…s, ko…ə, ko…d, ko…ə, ko…n...
5846    [wɪn, wɪ…d, wɪ…w, wɪ…ɝ, wɪ…d, w…nd, w…n…w, w…n...
2386    [ɡɔɹ, ɡɔ…d, ɡɔ…ʒ, ɡɔ…ə, ɡɔ…s, ɡ…ɹd, ɡ…ɹ…ʒ, ɡ…ɹ...
1325    [koʊ, ko…v, ko…ɑ, ko…ɫ, ko…ə, ko…n, ko…t, k…ʊv...
2973    [ɪnv, ɪn…ə, ɪn…ɫ, ɪn…ə, ɪn…d, ɪ…və, ɪ…v…ɫ, ɪ…v...
                              ...                        
2930    [ɪnt, ɪn…ɝ, ɪn…ə, ɪn…s, ɪn…t, ɪn…ɪ, ɪn…ŋ, ɪ…tɝ...
5164    [θiə, θi…ɫ, θi…ɑ, θi…d, θi…ʒ, θi…ɪ, θi…k, θi…ə...
497     [əve, əv…ɪ, əv…ɫ, əv…ə, əv…b, əv…ə, əv…ɫ, ə…eɪ...
3581    [nɝv, nɝ…i, n…vi, ɝvi, nɝ, n…v, n…i, ɝv, ɝ…i, ...
330     [ænt, æn…i, æn…v, æn…a, æn…ɪ, æn…ɹ, æn…ə, æn…ɫ...
Name: sn_skippy3gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [81]:
## sound skippy 4grams
add_ngram_to_df(df, n_for_ngram = 4, var_prefix = "sn_", ngram_is_skippy = True, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 2
source_var: sn_1gram
supplement_var: sn_skippy3gram
added_var: sn_skippy4gram
1078    [koʊɪ, koʊ…n, koʊ…s, koʊ…ə, koʊ…d, koʊ…ə, koʊ…...
5846    [wɪnd, wɪn…w, wɪn…ɝ, wɪn…d, wɪ…dw, wɪ…d…ɝ, wɪ…...
2386    [ɡɔɹd, ɡɔɹ…ʒ, ɡɔɹ…ə, ɡɔɹ…s, ɡɔ…dʒ, ɡɔ…d…ə, ɡɔ…...
1325    [koʊv, koʊ…ɑ, koʊ…ɫ, koʊ…ə, koʊ…n, koʊ…t, ko…v...
2973    [ɪnvə, ɪnv…ɫ, ɪnv…ə, ɪnv…d, ɪn…əɫ, ɪn…ə…ə, ɪn…...
                              ...                        
2930    [ɪntɝ, ɪnt…ə, ɪnt…s, ɪnt…t, ɪnt…ɪ, ɪnt…ŋ, ɪn…ɝ...
5164    [θiəɫ, θiə…ɑ, θiə…d, θiə…ʒ, θiə…ɪ, θiə…k, θiə…...
497     [əveɪ, əve…ɫ, əve…ə, əve…b, əve…ə, əve…ɫ, əv…ɪ...
3581    [nɝvi, nɝv, nɝ…i, n…vi, ɝvi, nɝ, n…v, n…i, ɝv,...
330     [ænti, ænt…v, ænt…a, ænt…ɪ, ænt…ɹ, ænt…ə, ænt…...
Name: sn_skippy4gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [82]:
## sound skippy 5grams
add_ngram_to_df(df, n_for_ngram = 5, var_prefix = "sn_", ngram_is_skippy = True, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 3
source_var: sn_1gram
supplement_var: sn_skippy4gram
added_var: sn_skippy5gram
1078    [koʊɪn, koʊɪ…s, koʊɪ…ə, koʊɪ…d, koʊɪ…ə, koʊɪ…n...
5846    [wɪndw, wɪnd…ɝ, wɪnd…d, wɪn…wɝ, wɪn…w…d, wɪn…ɝ...
2386    [ɡɔɹdʒ, ɡɔɹd…ə, ɡɔɹd…s, ɡɔɹ…ʒə, ɡɔɹ…ʒ…s, ɡɔɹ…ə...
1325    [koʊvɑ, koʊv…ɫ, koʊv…ə, koʊv…n, koʊv…t, koʊ…ɑɫ...
2973    [ɪnvəɫ, ɪnvə…ə, ɪnvə…d, ɪnv…ɫə, ɪnv…ɫ…d, ɪnv…ə...
                              ...                        
2930    [ɪntɝə, ɪntɝ…s, ɪntɝ…t, ɪntɝ…ɪ, ɪntɝ…ŋ, ɪnt…əs...
5164    [θiəɫɑ, θiəɫ…d, θiəɫ…ʒ, θiəɫ…ɪ, θiəɫ…k, θiəɫ…ə...
497     [əveɪɫ, əveɪ…ə, əveɪ…b, əveɪ…ə, əveɪ…ɫ, əve…ɫə...
3581          [nɝvi, nɝv, nɝ…i, n…vi, ɝvi, n…v, n…i, ɝ…i]
330     [æntiv, ænti…a, ænti…ɪ, ænti…ɹ, ænti…ə, ænti…ɫ...
Name: sn_skippy5gram, Length: 4453, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/3334243955.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [83]:
## check df
dropped_vars = [ 'sp_size', 'hyphen', 'period', 'sn_size' ]
if term_class == 'spell':
    extra = [ 'sn_1gram', 'sn_2gram', 'sn_3gram', 'sn_4gram',
             'sn_skippy2gram', 'sn_skippy3gram', 'sn_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]
else:
    extra = [ 'sp_1gram', 'sp_2gram', 'sp_3gram', 'sp_4gram',
             'sp_skippy2gram', 'sp_skippy3gram', 'sp_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]    
#
df[target_vars]

,spell,sound,sp_1gram,sp_2gram,sp_3gram,sp_4gram,sp_5gram,sp_skippy2gram,sp_skippy3gram,sp_skippy4gram,sp_skippy5gram,sn_skippy5gram
1078,coincident,koʊɪnsədənt,"[c, o, i, n, c, i, d, e, n, t]","[co, oi, in, nc, ci, id, de, en, nt, c, o, i, ...","[coi, oin, inc, nci, cid, ide, den, ent, co, o...","[coin, oinc, inci, ncid, cide, iden, dent, coi...","[coinc, oinci, incid, ncide, ciden, ident, coi...","[co, c…i, c…n, c…c, c…i, c…d, c…e, c…n, c…t, o...","[coi, co…n, co…c, co…i, co…d, co…e, co…n, co…t...","[coin, coi…c, coi…i, coi…d, coi…e, coi…n, coi…...","[coinc, coin…i, coin…d, coin…e, coin…n, coin…t...","[koʊɪn, koʊɪ…s, koʊɪ…ə, koʊɪ…d, koʊɪ…ə, koʊɪ…n..."
5846,windward,wɪndwɝd,"[w, i, n, d, w, a, r, d]","[wi, in, nd, dw, wa, ar, rd, w, i, n, d, w, a,...","[win, ind, ndw, dwa, war, ard, wi, in, nd, dw,...","[wind, indw, ndwa, dwar, ward, win, ind, ndw, ...","[windw, indwa, ndwar, dward, wind, indw, ndwa,...","[wi, w…n, w…d, w…w, w…a, w…r, w…d, in, i…d, i…...","[win, wi…d, wi…w, wi…a, wi…r, wi…d, w…nd, w…n…...","[wind, win…w, win…a, win…r, win…d, wi…dw, wi…d...","[windw, wind…a, wind…r, wind…d, win…wa, win…w…...","[wɪndw, wɪnd…ɝ, wɪnd…d, wɪn…wɝ, wɪn…w…d, wɪn…ɝ..."
2386,gorgeous,ɡɔɹdʒəs,"[g, o, r, g, e, o, u, s]","[go, or, rg, ge, eo, ou, us, g, o, r, g, e, o,...","[gor, org, rge, geo, eou, ous, go, or, rg, ge,...","[gorg, orge, rgeo, geou, eous, gor, org, rge, ...","[gorge, orgeo, rgeou, geous, gorg, orge, rgeo,...","[go, g…r, g…g, g…e, g…o, g…u, g…s, or, o…g, o…...","[gor, go…g, go…e, go…o, go…u, go…s, g…rg, g…r…...","[gorg, gor…e, gor…o, gor…u, gor…s, go…ge, go…g...","[gorge, gorg…o, gorg…u, gorg…s, gor…eo, gor…e…...","[ɡɔɹdʒ, ɡɔɹd…ə, ɡɔɹd…s, ɡɔɹ…ʒə, ɡɔɹ…ʒ…s, ɡɔɹ…ə..."
1325,covalent,koʊvɑɫənt,"[c, o, v, a, l, e, n, t]","[co, ov, va, al, le, en, nt, c, o, v, a, l, e,...","[cov, ova, val, ale, len, ent, co, ov, va, al,...","[cova, oval, vale, alen, lent, cov, ova, val, ...","[coval, ovale, valen, alent, cova, oval, vale,...","[co, c…v, c…a, c…l, c…e, c…n, c…t, ov, o…a, o…...","[cov, co…a, co…l, co…e, co…n, co…t, c…va, c…v…...","[cova, cov…l, cov…e, cov…n, cov…t, co…al, co…a...","[coval, cova…e, cova…n, cova…t, cov…le, cov…l…...","[koʊvɑ, koʊv…ɫ, koʊv…ə, koʊv…n, koʊv…t, koʊ…ɑɫ..."
2973,invalid,ɪnvəɫəd,"[i, n, v, a, l, i, d]","[in, nv, va, al, li, id, i, n, v, a, l, i, d]","[inv, nva, val, ali, lid, in, nv, va, al, li, ...","[inva, nval, vali, alid, inv, nva, val, ali, l...","[inval, nvali, valid, inva, nval, vali, alid, ...","[in, i…v, i…a, i…l, i…i, i…d, nv, n…a, n…l, n…...","[inv, in…a, in…l, in…i, in…d, i…va, i…v…l, i…v...","[inva, inv…l, inv…i, inv…d, in…al, in…a…i, in…...","[inval, inva…i, inva…d, inv…li, inv…l…d, inv…i...","[ɪnvəɫ, ɪnvə…ə, ɪnvə…d, ɪnv…ɫə, ɪnv…ɫ…d, ɪnv…ə..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2930,interesting,ɪntɝəstɪŋ,"[i, n, t, e, r, e, s, t, i, n, g]","[in, nt, te, er, re, es, st, ti, in, ng, i, n,...","[int, nte, ter, ere, res, est, sti, tin, ing, ...","[inte, nter, tere, eres, rest, esti, stin, tin...","[inter, ntere, teres, erest, resti, estin, sti...","[in, i…t, i…e, i…r, i…e, i…s, i…t, i…i, i…n, i...","[int, in…e, in…r, in…e, in…s, in…t, in…i, in…n...","[inte, int…r, int…e, int…s, int…t, int…i, int…...","[inter, inte…e, inte…s, inte…t, inte…i, inte…n...","[ɪntɝə, ɪntɝ…s, ɪntɝ…t, ɪntɝ…ɪ, ɪntɝ…ŋ, ɪnt…əs..."
5164,theological,θiəɫɑdʒɪkəɫ,"[t, h, e, o, l, o, g, i, c, a, l]","[th, he, eo, ol, lo, og, gi, ic, ca, al, t, h,...","[the, heo, eol, olo, log, ogi, gic, ica, cal, ...","[theo, heol, eolo, olog, logi, ogic, gica, ica...","[theol, heolo, eolog, ologi, logic, ogica, gic...","[th, t…e, t…o, t…l, t…o, t…g, t…i, t…c, t…a, t...","[the, th…o, th…l, th…o, th…g, th…i, th…c, th…a...","[theo, the…l, the…o, the…g, the…i, the…c, the…...","[theol, theo…o, theo…g, theo…i, theo…c, theo…a...","[θiəɫɑ, θiəɫ…d, θiəɫ…ʒ, θiəɫ…ɪ, θiəɫ…k, θiəɫ…ə..."
497,available,əveɪɫəbəɫ,"[a, v, a, i, l, a, b, l, e]","[av, va, ai, il, la, ab, bl, le, a, v, a, i, l...","[ava, vai, ail, ila, lab, abl, ble, av, va, ai...","[

# Analysis

In [84]:
## select data type and define doc_dict
import random
if "sp_" in term_type:
    base_type = "spell"
else:
    base_type = "sound"
doc_dict = { i: x for i, x in enumerate(df[base_type]) }
## check
random.sample(doc_dict.items(), 10)

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_6815/1181347170.py:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(doc_dict.items(), 10)


[(2880, 'revealing'),
 (263, 'alphabetic'),
 (2438, 'rectal'),
 (1791, 'dispirited'),
 (4341, 'conforming'),
 (51, 'tangerine'),
 (4021, 'sounding'),
 (28, 'oiled'),
 (3589, 'fathomable'),
 (128, 'unheard')]

In [85]:
## select bots for analysis
enable_term_change = False # if you want to change term_type to save time and energy
if enable_term_change:
	term_type = 'sp_skippy4gram'
	print(f"(changed) term_type: {term_type}")

## bot stands for 'bag-of-terms', a generalization of 'bag-of-words'
bots = [ x for x in df[term_type] if len(x) > min_bot_size ] # Crucially
import random
random.sample(bots, 1)

[['chat',
  'cha…t',
  'cha…y',
  'ch…tt',
  'ch…t…y',
  'ch…ty',
  'c…att',
  'c…at…y',
  'c…a…ty',
  'c…tty',
  'hatt',
  'hat…y',
  'ha…ty',
  'h…tty',
  'atty',
  'cha',
  'ch…t',
  'ch…t',
  'ch…y',
  'c…at',
  'c…a…t',
  'c…a…y',
  'c…tt',
  'c…t…y',
  'c…ty',
  'hat',
  'ha…t',
  'ha…y',
  'h…tt',
  'h…t…y',
  'h…ty',
  'att',
  'at…y',
  'a…ty',
  'tty',
  'ch',
  'c…a',
  'c…t',
  'c…t',
  'c…y',
  'ha',
  'h…t',
  'h…t',
  'h…y',
  'at',
  'a…t',
  'a…y',
  'tt',
  't…y',
  'ty']]

In [86]:
## generate dictionary
from gensim.corpora import Dictionary
diction = Dictionary(bots)
print(diction)
#apply_term_filtering = False
if apply_term_filtering:
    print(f"term filtering applied")
    diction.filter_extremes(no_below = term_minfreq, no_above = abuse_threshold)
else:
    print(f"term filtering not applied")
## check
print(diction)
## generate DTM
corpus = [ diction.doc2bow(bot) for bot in bots if len(bot) > min_bot_size ] # Crucially

Dictionary<198895 unique tokens: ['ci', 'cid', 'cide', 'cid…n', 'cid…t']...>
term filtering applied
Dictionary<97078 unique tokens: ['ci', 'cid', 'cide', 'cid…n', 'cid…t']...>


In [87]:
## HDP (n_topics = 15)
import gensim.models
import pyLDAvis.gensim

max_n_topics = 15
hdp15 = gensim.models.HdpModel(corpus, diction, T = max_n_topics, random_state = 1)
vis_data15 = pyLDAvis.gensim.prepare(hdp15, corpus, diction)
pyLDAvis.display(vis_data15)

In [88]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data15, vis_output)

In [89]:
## topic investigation
import numpy as np
import HDP_helper
reload_module = False
if reload_module:
    import importlib
    importlib.reload(HDP_helper)

target_hdp = hdp15
documents_topics = np.zeros([target_hdp.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in target_hdp[c]:
        documents_topics[topic_id][doc_id] = prob

target_hdp.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = target_hdp.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.002 * e…e + 0.001 * a…e + 0.001 * n…e + 0.001 * i…e + 0.001 * r…e + 0.001 * o…e + 0.001 * c…e + 0.001 * ed + 0.001 * e…t + 0.001 * e…i + 0.001 * s…e + 0.001 * e…l + 0.001 * e…a + 0.001 * a…l + 0.001 * u…e + 0.001 * le + 0.001 * l…e + 0.001 * e…d + 0.001 * e…s + 0.001 * r…d + 0.001 * i…l + 0.001 * p…e + 0.001 * o…i + 0.001 * n…i + 0.001 * n…l + 0.001 * n…d + 0.001 * e…n + 0.001 * b…e + 0.001 * c…n + 0.001 * t…e + 0.001 * o…t + 0.001 * e…r + 0.001 * a…d + 0.001 * in + 0.001 * o…s + 0.001 * e…ed + 0.001 * c…l + 0.001 * r…l + 0.001 * n…a + 0.001 * c…t + 0.001 * r…i + 0.001 * o…d + 0.001 * o…a + 0.001 * te + 0.001 * c…i + 0.001 * re + 0.001 * r…n + 0.001 * o…n + 0.0 * r…t + 0.0 * n…t + 0.0 * a…i + 0.0 * c…d + 0.0 * u…d + 0.0 * c…a + 0.0 * al + 0.0 * n…r + 0.0 * a…le + 0.0 * o…l + 0.0 * en + 0.0 * e…o
nonzero count:  2844
	0.9987: constricted
	0.9987: continental
	0.9987: inferential
	0.9987: complicated
	0.9987: uncorrected
	0.9987: incontinent
	0.9987: contentious
	0.9987: un

In [90]:
## save topic structures
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp15.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_save, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
hdp15_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp15_topics_out, header = False, index = None)

In [91]:
## HDP (n_topics = 45)
import gensim.models
import pyLDAvis.gensim

max_n_topics = 45
hdp45 = gensim.models.HdpModel(corpus, diction, T = max_n_topics, random_state = 1)
vis_data45 = pyLDAvis.gensim.prepare(hdp45, corpus, diction)
pyLDAvis.display(vis_data45)

In [92]:
## save LDAvis output as a html file
lang_dir_name = target_lang_dict[target_lang_key].split()[0]
save_LDAvis = True
if save_LDAvis:
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data45, vis_output)

In [93]:
## save topic structures
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp45.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_save, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp45_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp45_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp45_topics_out, header = False, index = None)

In [94]:
## topic investigation
import numpy as np
import HDP_helper

target_hdp = hdp45
documents_topics = np.zeros([target_hdp.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in target_hdp[c]:
        documents_topics[topic_id][doc_id] = prob
#
target_hdp.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = target_hdp.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: {len(probs.nonzero()[0])}")
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.002 * a…e + 0.002 * e…e + 0.001 * n…e + 0.001 * i…e + 0.001 * o…e + 0.001 * ed + 0.001 * r…e + 0.001 * c…e + 0.001 * a…l + 0.001 * s…e + 0.001 * e…t + 0.001 * t…e + 0.001 * e…d + 0.001 * e…i + 0.001 * l…e + 0.001 * u…e + 0.001 * r…d + 0.001 * e…a + 0.001 * a…d + 0.001 * le + 0.001 * e…l + 0.001 * o…t + 0.001 * a…i + 0.001 * te + 0.001 * b…e + 0.001 * t…d + 0.001 * e…s + 0.001 * o…s + 0.001 * n…d + 0.001 * p…e + 0.001 * o…i + 0.001 * a…a + 0.001 * e…ed + 0.001 * o…d + 0.001 * e…r + 0.001 * n…l + 0.001 * a…ed + 0.001 * in + 0.001 * o…a + 0.001 * u…d + 0.001 * i…l + 0.001 * c…l + 0.001 * a…le + 0.001 * n…i + 0.001 * i…d + 0.001 * r…i + 0.001 * c…n + 0.001 * er + 0.001 * c…d + 0.001 * e…n + 0.001 * n…r + 0.001 * r…l + 0.001 * o…l + 0.001 * n…a + 0.001 * r…t + 0.001 * n…t + 0.001 * a…t + 0.001 * c…t + 0.001 * r…n + 0.001 * r…a
nonzero count: 2447
	0.9988: complicated
	0.9988: uncorrected
	0.9987: contentious
	0.9987: uncontested
	0.9987: unconnected
	0.9987: compensated
	0.998

In [95]:
## HDP (n_topics = 90)
import gensim.models
import pyLDAvis.gensim

max_n_topics = 90
hdp90 = gensim.models.HdpModel(corpus, diction, T = max_n_topics,
                               random_state = 1)
vis_data90 = pyLDAvis.gensim.prepare(hdp90, corpus, diction)
pyLDAvis.display(vis_data90)

In [96]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data90, vis_output)

In [97]:
## save topic structures
hdp_topics = hdp90.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_save, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }

## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
hdp90_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp90_topics_out, header = False, index = None)

In [98]:
## topic investigation
import numpy as np
import HDP_helper

target_hdp = hdp90
documents_topics = np.zeros([target_hdp.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in target_hdp[c]:
        documents_topics[topic_id][doc_id] = prob

## investigate topics
target_hdp.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    #topic_encoding = ", ".join(hdp.show_topic(topic_id))
    topic_t = target_hdp.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.002 * a…e + 0.002 * e…e + 0.002 * i…e + 0.001 * n…e + 0.001 * ed + 0.001 * r…e + 0.001 * o…e + 0.001 * e…t + 0.001 * s…e + 0.001 * c…e + 0.001 * e…i + 0.001 * a…i + 0.001 * a…l + 0.001 * t…e + 0.001 * e…a + 0.001 * u…e + 0.001 * e…d + 0.001 * e…l + 0.001 * a…d + 0.001 * o…i + 0.001 * l…e + 0.001 * r…d + 0.001 * i…l + 0.001 * i…t + 0.001 * te + 0.001 * in + 0.001 * i…d + 0.001 * i…a + 0.001 * r…i + 0.001 * t…d + 0.001 * n…d + 0.001 * a…t + 0.001 * le + 0.001 * b…e + 0.001 * n…i + 0.001 * i…i + 0.001 * p…e + 0.001 * e…n + 0.001 * n…l + 0.001 * a…a + 0.001 * r…t + 0.001 * o…t + 0.001 * e…ed + 0.001 * r…l + 0.001 * a…ed + 0.001 * o…a + 0.001 * o…d + 0.001 * r…a + 0.001 * c…l + 0.001 * e…r + 0.001 * n…a + 0.001 * al + 0.001 * r…n + 0.001 * n…t + 0.001 * er + 0.001 * c…n + 0.001 * c…i + 0.001 * o…l + 0.001 * n…r + 0.001 * u…d
nonzero count:  2921
	0.9990: complicated
	0.9990: uncorrected
	0.9990: uncompleted
	0.9990: contestable
	0.9990: compensable
	0.9990: inseparable
	0.9990